# Grid Search

In [1]:
import numpy as np
import tensorflow as tf
import itertools
import pandas as pd
# Own library
import test2
# Needed to clean memory
import gc

# To modify scipts remotly
%load_ext autoreload
%autoreload 2

In [2]:
np.random.seed(94)
tf.random.set_seed(94)
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_test = x_test / 255.0
x_test, y_test = tf.constant(x_test), tf.constant(y_test)

n = 2
x_test = x_test[:n]
y_test = tf.squeeze(tf.one_hot(y_test[:n], 10))

In [3]:
path = 'data/model_cifar.h5'
model = tf.keras.models.load_model(path)

In [4]:
# Parameter space
list_eta = np.append(np.power(0.1, [1, 2, 3, 4]), 1/255)
list_epsilon = np.append(np.power(0.1, [0, 1, 2, 3, 4]), 1/255)
list_n_steps = np.arange(1, 10, 2)
list_losses = [tf.keras.losses.MeanSquaredError(),
               tf.keras.losses.CategoricalCrossentropy(),
               tf.keras.losses.KLDivergence()]

list_methods = [test2.pgd_infinity, test2.pgd_ininity_random]

# Generate parameters grid
iter_parameters_grid = itertools.product(list_methods, list_eta, list_epsilon, list_n_steps, list_losses)


# Names of methods
list_method_names = ['pgd_infinity', 'pgd_random']

In [ ]:
# Compute different attack sets

nested_accuracies = []
for method, eta, eps, n_steps, loss in iter_parameters_grid:
    if eta >= eps:
        # Does not make sense to test this parameters config
        nested_accuracies.append([np.nan, np.nan])
        continue
    
    x_adv = method(x_test, y_test, model, loss_fn=loss, eta=eta, eps=eps, n_steps=n_steps)
    # Re-scaling image (they are normalized)
    x_adv = tf.clip_by_value(x_adv, 0, 1)
    list_eval = model.evaluate(x=x_adv, y=y_test)
    # Clear memory
    gc.collect()
    tf.keras.backend.clear_session()
    del x_adv
      
    nested_accuracies.append(list_eval)

2/2 [==============================] - 0s 2ms/sample - loss: 0.2290 - accuracy: 1.0000


2/2 [==============================] - 0s 7ms/sample - loss: 0.6452 - accuracy: 0.5000


In [ ]:
index = pd.MultiIndex.from_product([list_method_names, list_eta, list_epsilon],
                                   names=['method', 'tau', 'epsilon'])

In [ ]:
# Graphs when step size is big
index = pd.MultiIndex.from_product([list_method_names, list_eta, list_epsilon,
                                    list_n_steps, [loss.name for loss in list_losses]],
                                   names=['method', 'eta', 'epsilon', 'n_steps', 'loss_name'])

df_results = pd.DataFrame(data=nested_accuracies,
                          columns=['loss', 'accuracy'],
                          index=index)

In [ ]:
df_results.dropna()

In [ ]:
path = './data/df_results.pickle'
import pickle
with open(path, 'wb') as f:
    pickle.dump(df_results, f)